In [1]:
import torch

## Convolutional layer

In [2]:
in_channels, out_channels = 5, 10
width, height = 100,100
kernel_size = 3
batch_size = 1

input_data = torch.randn(batch_size,
                        in_channels,
                        width,
                        height)

conv_layer = torch.nn.Conv2d(in_channels,
                           out_channels,
                           kernel_size = kernel_size)

output = conv_layer(input_data)

print(input_data.shape)
print(output.shape)
print(conv_layer.weight.shape)

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


## padding

In [6]:
input = [3,4,6,5,7,
         2,4,6,8,2,
         1,6,7,8,4,
         9,7,4,6,2,
         3,7,5,4,1]
input = torch.Tensor(input).view(1,1,5,5) # batch=1, channel=1, width=5, height=5

conv_layer = torch.nn.Conv2d(1, 1, kernel_size = 3, padding = 1, bias = False) # input_channel=1, output_channel=1

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3) # output_channel=1, input_channel=1, width=1, height=1
conv_layer.weight.data = kernel.data

output = conv_layer(input)
print(output)
print(output.shape)

tensor([[[[ 91., 168., 224., 215., 127.],
          [114., 211., 295., 262., 149.],
          [192., 259., 282., 214., 122.],
          [194., 251., 253., 169.,  86.],
          [ 96., 112., 110.,  68.,  31.]]]], grad_fn=<ThnnConv2DBackward0>)
torch.Size([1, 1, 5, 5])


## Stride 步长

In [7]:
input = [3,4,6,5,7,
         2,4,6,8,2,
         1,6,7,8,4,
         9,7,4,6,2,
         3,7,5,4,1]
input = torch.Tensor(input).view(1,1,5,5) 

conv_layer = torch.nn.Conv2d(1, 1, kernel_size = 3, stride = 2, bias = False) # stride=2

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3) 
conv_layer.weight.data = kernel.data

output = conv_layer(input)
print(output)
print(output.shape)

tensor([[[[211., 262.],
          [251., 169.]]]], grad_fn=<ThnnConv2DBackward0>)
torch.Size([1, 1, 2, 2])


## Maxpooling

In [8]:
input = [3,4,6,5,
         2,4,6,8,
         1,6,7,8,
         9,7,4,6,
        ]
input = torch.Tensor(input).view(1, 1, 4, 4)

maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2)

output = maxpooling_layer(input)
print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


## homework

In [9]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [10]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='../dataset/mnist/',
                               train=True,
                               download=False, # True
                               transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist/',
                              train=False,
                              download=False, # 第一次下载True
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

In [11]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size = 5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320,10)
    
    def forward(self, x):
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # flatten
        x = self.fc(x)
        return x

model = Net()

In [12]:
criterion = torch.nn.CrossEntropyLoss() # 交叉熵损失
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #带有冲量，目的能冲过鞍面和部分局部最优

In [13]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # predicted是一行最大值的下标,dim=1表示沿着列的维度
            total += labels.size(0) # 当前batch里数据标签的数目，即batch_size
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [14]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.656
[1,   600] loss: 0.184
[1,   900] loss: 0.145
Accuracy on test set: 96 %
[2,   300] loss: 0.105
[2,   600] loss: 0.101
[2,   900] loss: 0.093
Accuracy on test set: 97 %
[3,   300] loss: 0.076
[3,   600] loss: 0.075
[3,   900] loss: 0.077
Accuracy on test set: 97 %
[4,   300] loss: 0.062
[4,   600] loss: 0.062
[4,   900] loss: 0.065
Accuracy on test set: 98 %
[5,   300] loss: 0.059
[5,   600] loss: 0.056
[5,   900] loss: 0.051
Accuracy on test set: 98 %
[6,   300] loss: 0.047
[6,   600] loss: 0.051
[6,   900] loss: 0.050
Accuracy on test set: 98 %
[7,   300] loss: 0.045
[7,   600] loss: 0.046
[7,   900] loss: 0.044
Accuracy on test set: 98 %
[8,   300] loss: 0.043
[8,   600] loss: 0.041
[8,   900] loss: 0.042
Accuracy on test set: 98 %
[9,   300] loss: 0.036
[9,   600] loss: 0.038
[9,   900] loss: 0.041
Accuracy on test set: 98 %
[10,   300] loss: 0.036
[10,   600] loss: 0.034
[10,   900] loss: 0.039
Accuracy on test set: 98 %
